# Consultancy report analysis
## Motorbike scrambling track
By Gillian Kennedy (B00805416)

## Introduction
There is a planning permission proposal of a motorbike scrambling track in the Binevenagh area. Binevenagh is located in Northern Ireland in County Londonderry along the northwest coast. The purpose of the code created is to look at potential impacts the motorbike scrambling track may have on local residents and the environment. 

## Aim 
Analyse the potential impact of the motorbike scrambling track in the Binevenagh area.

## Objectives
1. Analyse potential impact on local residents
2. Analyse potential impact on environment

## Data Provided
- NI_outline (Shapefile displaying outline of Northern Ireland)
- ASSI (Shapefile displaying Areas of Special Scientific Interest)
- AONB (Shapefile displaying Areas of Outstanding Natural Beauty) 
- NW_coast_Land_Cover (Shapefile displaying land cover over the northwest of Northern Ireland)
- Gazeteer (Shapefile displaying points representing settlements across Northern Ireland) 
- Settlements (Shapefile displaying polygon outlines of settlements acroos Northern Ireland)
- Census_areas (Shapefile displaying census population data over the Binevenagh area)
- Children_0-7 (CSV file containing the number of children under 7 in the Binevenagh area)
- Binevenagh Pointer data (CSV file containing point data of buildings in the Binevenagh area)
- New buildings (CSV file containing GPS points of new buildings in the Binevenagh area)

## Preparing data for analysis

In [ ]:
#use figures interactively
%matplotlib widget

#importing packages
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import folium
from shapely.geometry import Point, LineString, Polygon
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature

plt.ion() #make plotting interactive

In [ ]:
# Loading all shapefile data
outline = gpd.read_file(os.path.abspath('data_files/NI_outline.shp'))
settlements = gpd.read_file(os.path.abspath('data_files/settlements_poly.shp'))
gazeteer = gpd.read_file(os.path.abspath('data_files/Gazeteer.shp'))
assi = gpd.read_file(os.path.abspath('data_files/ASSI.shp'))
aonb = gpd.read_file(os.path.abspath('data_files/AONB.shp'))
census = gpd.read_file(os.path.abspath('data_files/census_areas.shp'))
landcover = gpd.read_file(os.path.abspath('data_files/NW_coast_Land_Cover.shp'))

In [ ]:
# create a Universal Transverse Mercator reference system to transform the data
ni_utm = ccrs.UTM(29) 

In [ ]:
# Ensuring all data is on the same projection
outline = outline.to_crs(ni_utm)
settlements = settlements.to_crs(ni_utm)
gazeteer = gazeteer.to_crs(ni_utm)
assi = assi.to_crs(ni_utm)
aonb = aonb.to_crs(ni_utm)
census = census.to_crs(ni_utm)
landcover = landcover.to_crs(ni_utm)

In [ ]:
#Creating a map figure
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ni_utm)

In [ ]:
# Adding layers to the map figure
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to black
                                 facecolor='w', # set the facecolor to white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

assi_feat = ShapelyFeature(assi['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='pink', # set the edgecolor to be pink
                            facecolor='pink', # set the facecolor to be pink
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=2) # second layer to be drawn on map
ax.add_feature(assi_feat) # add the feature to the map figure

aonb_feat = ShapelyFeature(aonb['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='purple', # set the edgecolor to be purple
                            facecolor='purple', # set the facecolor to be purple
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=3) # third layer to be drawn on map
ax.add_feature(aonb_feat) # add the feature to the map figure

landcover_feat = ShapelyFeature(landcover['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='green', # set the edgecolor to be green
                            facecolor='green', # set the facecolor to be green
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=4) # fourth layer to be drawn on map
ax.add_feature(landcover_feat) # add the feature to the map figure

census_feat = ShapelyFeature(census['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='orange', # set the edgecolor to be orange
                            facecolor='orange', # set the facecolor to be orange
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=5) # fifth layer to be drawn on map
ax.add_feature(census_feat) # add the feature to the map figure

settlements_feat = ShapelyFeature(settlements['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='gray', # set the edgecolor to be gray
                            facecolor='gray', # set the facecolor to be gray
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=6) # sixth layer to be drawn on map
ax.add_feature(settlements_feat) # add the feature to the map figure

gazeteer_feat = ax.scatter(gazeteer.geometry.x, gazeteer.geometry.y, # first argument is the geometry
                            transform=ccrs.UTM(zone=29, southern_hemisphere=False), # second argument is the CRS
                            color='yellow', # set color to yellow
                            s= 10, # set size to 10
                            zorder=7) # seveth layer to be drawn on map

xmin, ymin, xmax, ymax = outline.total_bounds # using the boundary of the outline shapefile feature, zoom the map to area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) 

# Show map figure
fig


As can be seen from the two maps, census areas and landcover are they only layers which do not cover the extent of Northern Ireland as they have already been reduced to cover the area of Binevenagh where the motorbike track will be located. Since we are focusing on the area surrounding the track we should clip all the layers to the extent of the study area. 

In [ ]:
# Creating a polygon to represent the study area
study_area = Polygon([(-7.030049, 55.041508), (-7.026122, 55.203187), (-6.633537, 55.19942), (-6.639045, 55.037764)])

#creating geodataframe for study area
study_area_gdf = gpd.GeoDataFrame({'geometry': [study_area]}, crs=4326).to_crs(ni_utm)

# Clearing map figure
ax.clear()

#Adding NI outline and study area to map to show where study is in Northern Ireland
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to be black
                                 facecolor='w', # set the facecolor to be white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

study_area_gdf.plot(ax=ax, color='none', edgecolor='forestgreen')

xmin, ymin, xmax, ymax = outline.total_bounds # using the boundary of the outline shapefile feature, zoom the map to area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) 

# Show map figure
fig

In [ ]:
# Clipping layers to study area extent
settlements_st = settlements.clip(study_area_gdf)
gazeteer_st = gazeteer.clip(study_area_gdf)
assi_st = assi.clip(study_area_gdf)
aonb_st = aonb.clip(study_area_gdf)

In [ ]:
# Clearing map figure
ax.clear()

#Adding layers to the map figure
outline.plot(ax=ax, color='none', edgecolor='black')

# Plot one the clipped layers
settlements_st.plot(ax=ax, color='gray', edgecolor='black')

xmin, ymin, xmax, ymax = study_area_gdf.total_bounds # using the boundary of the study area, zoom the map to our area of interest
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=ni_utm) 

# Show map figure
fig

In [ ]:
#Creating a point to represent the track centre
track_centre = Point(-6.833074, 55.143815)

#Creating a geodataframe for track centre point
track_gdf = gpd.GeoDataFrame({'geometry': [track_centre]}, crs=4326).to_crs(ni_utm)

# Plotting track centre
track_gdf.plot(ax=ax, color='red', markersize=50)

# Show the map figure
fig


### Preparing Building data

In [ ]:
df = pd.read_csv('data_files/Binevenagh_Pointer.csv') # read csv data

# create a new geodataframe
buildings = gpd.GeoDataFrame(df,
                             geometry=gpd.points_from_xy(df['X_COR'], df['Y_COR']), 
                             crs=29903).to_crs(ni_utm)

# show table
buildings.head()

In [ ]:
# Plotting building data to view
buildings.plot(ax=ax, color='darkblue', markersize=2)

# Show the map
fig

The Binevenagh Pointer data is out of date and new buildings have been constructed since the pointer data was collected. A local resident carried out a survey providing GPS points of the most recently built buildings in the area surrounding the motorbike track.

In [ ]:
# Find out how may buildings there was originally
print(f"number of buildings: {len(buildings)}")

In [ ]:
df2 = pd.read_csv('data_files/new_buildings.csv') # read csv data

# create a new geodataframe
new_buildings = gpd.GeoDataFrame(df2,
                             geometry=gpd.points_from_xy(df2['easting'], df2['northing']), 
                             crs=29903).to_crs(ni_utm)

# show table
new_buildings.head()

In [ ]:
# Find out how many new buildings there are
print(f"number of new buildings: {len(new_buildings)}")

In [ ]:
# Merge the original building data with the new building data
buildings_updated = gpd.GeoDataFrame(pd.concat([buildings, new_buildings], ignore_index=True))

# Ensure the new number of buildings is correct
print(f"number of new buildings: {len(buildings_updated)}")

In [ ]:
# show table
buildings_updated.head()

### Preparing landcover data

In [ ]:
# show table
landcover.head()

To find out which type and category belongs to which BHSUB number see the land cover data description pdf or land cover summary table word document found in the data_files folder.

In [ ]:
# define landcover types in a list
type = ['broad-leaved woodland', 'coniferous woodland', 'arable horticulture', 'improved grassland', 'neutral grass', 'calcareous grass', 'acid grass', 'bracken', 'dwarf shrub heath', 
        'open dwarf shrub heath', 'bog', 'water (inland)', 'inland bare ground', 'suburban/rural developed', 'continuous urban', 'supra-littoral sediment', 'littoral rock',
        'littoral sediment', 'saltmarsh', 'sea/estuary']

# BHSUB number that corresponds to the type of landcover
values = [1.1, 2.1, 4.2, 5.1, 6.1, 7.1, 8.1, 9.1, 10.1, 10.2, 12.1, 13.1, 16.1, 17.1, 17.2, 19.1, 20.1, 21.1, 21.2, 22.1]

# Category each landcover type falls into
category = ['woodland', 'woodland', 'agricultural (intensive)', 'agricultural (intensive)', 'agricultural (non-intensive)', 'agricultural (non-intensive)', 'agricultural (non-intensive)',
            'semi-natural', 'semi-natural', 'semi-natural', 'semi-natural', 'water', 'non-vegetated', 'built environment', 'built environment', 'non-vegetated', 'non-vegetated', 
            'non-vegetated', 'semi-natural', 'water']

landcover_type = dict(zip(values, type)) # create a dict of landcover value/type pairs
landcover_category = dict(zip(values, category)) # create a dict of landcover value/category pairs

In [ ]:
# Add landcover type and landcover category as columns to landcover shapefile
landcover["Landcover_Type"] = landcover["BHSUB"].map(landcover_type)
landcover["Landcover_Category"] = landcover["BHSUB"].map(landcover_category)

# show attribute table of landcover shapefile
landcover.head()

In [ ]:
# Clearing map figure
ax.clear()

#Adding layers to the map figure
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to be black
                                 facecolor='w', # set the facecolor to be white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

# setting colours for landcover
landcover_colors = ['lawngreen', 'palegreen', 'gray', 'orangered', 'saddlebrown', 'aqua', 'darkgreen']

landcover_eco_category = list(landcover.Landcover_Category.unique())
landcover_eco_category = [str(x) for x in landcover_eco_category]  # Convert everything to string
landcover_eco_category.sort()

for ii, name in enumerate(landcover_eco_category):
    landcover_feat = ShapelyFeature(landcover.loc[landcover["Landcover_Category"] == name, "geometry"],
                                    ni_utm,
                                    edgecolor='k',
                                    facecolor=landcover_colors[ii],
                                    linewidth=1,
                                    alpha=0.25)
    ax.add_feature(landcover_feat)

xmin, ymin, xmax, ymax = study_area_gdf.total_bounds # using the boundary of the study area, zoom the map to our area of interest
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=ni_utm) 

# Show map figure
fig

### Preparing census data

For the analysis section of the code we will want to know if the area surrounding the motorbike track has a high proportion of young children. As the noises from the motorbike track my affect the children at bedtime. So we need to join the children data to the census data.

In [ ]:
children = pd.read_csv('data_files/children_0-7.csv') # read csv data

# show table
children.head()

In [ ]:
#show table
census.head()

In [ ]:
# joining children data to census data
census_updated = census.merge(children, left_on='OA_CODE', right_on='OA')

# show table
census_updated.head()

## Analysing Data

There are many buildings surrounding the motorbike track and so many people that could potentially be affected by the noise produced by the track. Before assessing the impact the track could potntially have on local residents it might be of interest to see how far away the owner of the track lives. The owner of the track lives in a village called Articlave.

In [ ]:
# show table
gazeteer.head()

In [ ]:
# Selecting the village Articlave
articlave = gazeteer.loc[gazeteer['NAME'] == 'ARTICLAVE', 'geometry'].values[0]

# calculating the distance of Articlave village to the track
owner_distance = track_gdf.distance(articlave)

distance_value = owner_distance.loc[0]

print(f'Distance of track to owners house: {distance_value:.2f} meters')

There has been official complaints made about the motorbike track from local residents. The majority of complaints come from people living on the Altikeeragh Road, Ballyhackett Road, and Burrenmore Road. We can use these complaints to estimate how far the noise of the track travels in the area.

In [ ]:
# Calculating the distance of all buildings in the study area from the track
buildings["distance_to_track"] = buildings.geometry.distance(track_gdf.loc[0, 'geometry'])

# Selecting the roads where complaint were made from
complaints = buildings.loc[buildings['ROAD'].isin(['ALTIKEERAGH ROAD', 'BALLYHACKETT ROAD', 'BURRENMORE ROAD'])]

print(complaints[["ROAD", "distance_to_track"]])

As we can see from the results above most complaints are within 1.2km. However there have also been complaints made by holiday-makers staying at a caravan park at 56 Ballywoodock Road who say noise can be heard from the track in calm weather.

In [ ]:
# Selecting the caravan park the complaint was made from
complaints2 = buildings.loc[(buildings['ROAD'] == 'BALLYWOODOCK ROAD') & (buildings['BUILDING'] == 56)]

print(complaints2[["BUILDING", "ROAD", "distance_to_track"]])

From calculating the distance from the track to the caravn park we now know that the noise from the track can be heard over 2km away.
Another factor to help analyse the impact of the track on local residents is to calculate the population density of the surrounding area and the percentage of children in the surrounding area.

In [ ]:
# Rename the child data column
census_updated = census_updated.rename(columns={"CHILD_0-7": "CHILD_0_7"})

# show table
census_updated.head()

In [ ]:
# Function
def population_calculations(census_data, population, children):
    """
    Calculates area, population density, and percentage of children.

    Parameters:
    census_data: The polygon dataset.
    population : Column name containing total population values from the census.
    children : Column name containing number of children aged 0-7.

    Returns:
    census_results: Updated census_updated with new columns 'area', 'population_density', and 'children_percentage'.
    """
    # Calculate area
    census_data["area"] = census_data.geometry.area

    # Calculate population density
    census_data["pop_density"] = (census_data[population] / census_data["area"]) * 1_000_000

    # Calculate percentage of children
    census_data["pc_child"] = (census_data[children] / census_data[population]) * 100

    return census_data


census_data = population_calculations(census_updated, "POPN", "CHILD_0_7")

print(census_data[["POPN", "CHILD_0_7", "area", "pop_density", "pc_child"]])

In [ ]:
# Calculate area for landcover shapefile
landcover["area"] = landcover.geometry.area

# show table
landcover.head()

In [ ]:
# Creating a 2km buffer around the track centre
buffer_2km = track_gdf.buffer(2000)

# Plot buffer onto map figure
buffer_2km.plot(ax=ax, facecolor='none', edgecolor='k')

# Show map figure
fig

## Results

### Impact on local residents


#### Map depicting the location of buildings and population density of the study area

In [ ]:
# Plot folium map
m = buffer_2km.explore(fill=False,
                       color='black')
                       

census_updated.explore('pop_density', 
                       m=m,
                       cmap='RdYlBu',
                       legend_kwds={'caption': 'Population density'})

track_args = {
    'm' : m,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)

buildings_args = {
    'm' : m,
    'marker_type': 'circle_marker',
    'popup': True,
    'legend': False,
    'marker_kwds': { 'radius': 2, 'color': 'gray', 'fill': True, 'fill_color': 'gray', 'fillOpacity': 1.0}
}
    

buildings_updated.explore(**buildings_args)


# Show folium map
m

#### Number of buildings, and their use found within set distances of the motorbike scrambling track.

In [ ]:
# Creating buffers for set distances of 250m, 500m, and 1km
buffer_250m = track_gdf.buffer(250)
buffer_500m = track_gdf.buffer(500)
buffer_1km = track_gdf.buffer(1000)

# Clipping buildings within each buffer (250m. 500m, 1km, 2km)
buildings_250m = buildings_updated.clip(buffer_250m)
buildings_500m = buildings_updated.clip(buffer_500m)
buildings_1km = buildings_updated.clip(buffer_1km)
buildings_2km = buildings_updated.clip(buffer_2km)

# Function
def building_stats_calculation(buildings_clipped, buffer_zone):
    """
    Calculates count and percentage of residential, commercial, and unknown buildings within a buffer.

    Parameters:
    - buildings_clipped: Buildings dataset clipped to the buffer.
    - buffer_zone: Buffer distance

    Returns:
    - dict: Count and percentage of buildings by classification of the USE column.
    """
    total = len(buildings_clipped)
    residential = len(buildings_clipped[buildings_clipped["USE"] == "RESIDENTIAL"])
    commercial = len(buildings_clipped[buildings_clipped["USE"] == "COMMERCIAL"])
    unknown_class = len(buildings_clipped[buildings_clipped["USE"] == "UNKNOWN"])

    # Counting missing values if a building has not been classified as residential, commercial or unknown
    missing = 0
    if "USE" in buildings_clipped.columns:
        missing = len(buildings_clipped[buildings_clipped["USE"].isna()])
    
    # Add the buildings classified as unknown and any missing values
    unknown = unknown_class + missing

    return {
        f"{buffer_zone}m": {
            "Total Buildings": total,
            "Residential Count": residential,
            "Residential %": round((residential / total) * 100, 1) if total > 0 else 0,
            "Commercial Count": commercial,
            "Commercial %": round((commercial / total) * 100, 1) if total > 0 else 0,
            "Unknown Count": unknown,
            "Unknown %": round((unknown / total) * 100, 1) if total > 0 else 0
        }
    }


# Call the function for each distance
building_stats_250m = building_stats_calculation(buildings_250m, 250)
building_stats_500m = building_stats_calculation(buildings_500m, 500)
building_stats_1km = building_stats_calculation(buildings_1km, 1000)
building_stats_2km = building_stats_calculation(buildings_2km, 2000)

# Print results
print(building_stats_250m)
print(building_stats_500m)
print(building_stats_1km)
print(building_stats_2km)


In [ ]:
# Plot folium map
m2 = buffer_2km.explore(fill=False,
                       color='black')
                       

census_updated.explore('pc_child', 
                       m=m2,
                       cmap='RdYlBu',
                       legend_kwds={'caption': 'Percentage of children'})

track_args = {
    'm' : m2,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)

buildings_args = {
    'm' : m2,
    'marker_type': 'circle_marker',
    'popup': True,
    'legend': False,
    'marker_kwds': { 'radius': 2, 'color': 'gray', 'fill': True, 'fill_color': 'gray', 'fillOpacity': 1.0}
}
    

buildings_updated.explore(**buildings_args)


# Show folium map
m2

From the population density map it can be seen that within the 2km buffer surrounding the track population density is low. 

Building stats show that the majority of the buildings within 2km of the motorbike track are residential buildings with over 50% of the buildings in the area within 500m, 1000m and
2000m of the track categorized as residential properties. In total there are 160 buildings that could be affected by noise from the motorbike track.

From the percentage of children map it can be seen that within the 2km buffer the percentage of children ranges from average to below average. The noise pollution from the track
could potentially impact the sleep of children aged between 0 to 7 years old at bedtime within the 2km buffer.

## Impact on the environment

#### A map depicting the landcover of the study area 

In [ ]:
# print each unique landcover category
print(landcover["Landcover_Category"].unique())

In [ ]:
# Plot folium map
m3 = buffer_2km.explore(fill=False,
                       color='black')
                       


track_args = {
    'm' : m3,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)

# Applying colours to landcover categories
land_color_dict = {
    'woodland': '#006600', # Dark green
    'agricultural (intensive)': '#80ff80', # Light green
    'agricultural (non-intensive)': '#00e600', # Green
    'semi-natural': '#86592d', # Brown
    'water': '#0099ff', # Blue
    'built environment': '#ccccb3', # Gray/Brown
    'non-vegetated': '#ffffcc', # Pale yellow 
}

landcover.explore('Landcover_Category',
                  m=m3,
                  legend_kwds={'caption': 'Land Cover Types'}
)
                  

# Show folium map
m3

#### Percentage of ecologically valuable land (semi-natural and agricultural (nonintensive) land) found within set distances from the motorbike scrambling track

In [ ]:
# Clipping landcover within each buffer (250m. 500m, 1km, 2km)
landcover_250m = landcover.clip(buffer_250m)
landcover_500m = landcover.clip(buffer_500m)
landcover_1km = landcover.clip(buffer_1km)
landcover_2km = landcover.clip(buffer_2km)

# Function
def ecological_land_percentage(landcover_clip, buffer_zone):
    """
    Calculates the percentage of ecologically valuable land within a buffer.

    Parameters:
    - landcover_clip : Landcover data that has been clipped to the buffer.
    - buffer_zone : Buffer distance.

    Returns:
    - dict: Percentage of ecologically valuable land within the buffer distance.
    """
    total_area = float(landcover_clip["area"].sum())

    # Filter ecologically valuable land categories
    eco_land = landcover_clip[landcover_clip["Landcover_Category"].isin(["semi-natural", "agricultural (non-intensive)"])]
    eco_area = float(eco_land["area"].sum())

    # Calculate percentage
    eco_land_pc = float(eco_area / total_area) * 100 if total_area > 0 else 0

    return {
        f"{buffer_zone}m": {
            "Total Land Area": f"{round(total_area, 2)} m²",
            "Ecologically Valuable Land Area": f"{round(eco_area, 2)} m²",
            "Ecologically Valuable %": round(eco_land_pc, 1)
        }
    }

# Call the function for each buffer
landcover_stats_250m = ecological_land_percentage(landcover_250m, 250)
landcover_stats_500m = ecological_land_percentage(landcover_500m, 500)
landcover_stats_1km = ecological_land_percentage(landcover_1km, 1000)
landcover_stats_2km = ecological_land_percentage(landcover_2km, 2000)

# Print results
print(landcover_stats_250m)
print(landcover_stats_500m)
print(landcover_stats_1km)
print(landcover_stats_2km)

#### Maps depicting the Areas of Outstanding Natural Beauty (AONB) and Areas of Special Scientific Interest (ASSI) found within the Binevenagh area.

In [ ]:
# Adding coloum to each layer so category can be mapped
assi_st["Category"] = "ASSI"
aonb_st["Category"] = "AONB"

In [ ]:
# Plot folium map
m4 = buffer_2km.explore(fill=False,
                       color='black')
                       


track_args = {
    'm' : m4,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)


aonb_st.explore('Category',
    m=m4,
    cmap="YlOrBr",  
    legend= True,
    legend_kwds={'caption': 'Area of Outstanding Natural Beauty'}
)


# Show folium map
m4

In [ ]:
# Plot folium map
m5 = buffer_2km.explore(fill=False,
                       color='black')
                       


track_args = {
    'm' : m5,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)



assi_st.explore('Category',
    m=m5,
    cmap="Purples",  
    legend= True, 
    legend_kwds={'caption': 'Areas of Special Scientific Interest'}
)


# Show folium map
m5

#### Environmental impact

As seen from the landcover map motorbike track is situated on agricultural (intensive) land which has a low ecological value. Therefore, the construction the motorbike track would not have a considerably negative impact on the actual land the tack itself is located on. 

However, as can be seen in the landcover map the track is surrounded by semi-natural land and agricultural (non-intensive) land which have a high ecological value. Land with a high ecological value have habitats where there is possibly a diverse amount of plant, animal, and bird species. We known from complaints that noise pollution from the track can be heard within 2km of the track. From our calculations over 50% of the land within 2km of the track is considered to be of a high ecological value. Therefore, the proximity of high ecological value land means that the motorbike track may have a harmful environmental impact on the birds, animals, and plants within the habitats due to the pollution, noise, and disturbances caused by the track. 

The track is also located in an AONB and an ASSI falls into the 2km buffer of the track suggesting further evidence that the track may have a potential negative impact on the environment. 